In [ ]:
# initialization for my classroom
import os
from datetime import datetime as dt

def logfile(user=os.environ.get('JUPYTERHUB_USER') or 'jovyan'):
    prefix='/srv'
    if os.path.isdir(prefix) and os.access(prefix, os.W_OK):
        prefix+=('/'+user)
        if not os.path.isdir(prefix):
            os.makedirs(prefix)
    else:
        prefix='.'
    return prefix+'/'+dt.now().strftime('%Y%m%d')+'.log'

path=logfile()
#%logstop
%logstart -otq $path append

# [python - cannot override sys.excepthook - Stack Overflow](https://stackoverflow.com/questions/1261668/cannot-override-sys-excepthook/28758396)
# https://github.com/ipython/ipython/blob/e6432249582e05f438303ce73d082a0351bb383e/IPython/core/interactiveshell.py#L1952

import sys
import traceback
import IPython

try:
    _showtraceback
except NameError:
    _showtraceback=IPython.core.interactiveshell.InteractiveShell.showtraceback

try:
    _showsyntaxerror
except NameError:
    _showsyntaxerror=IPython.core.interactiveshell.InteractiveShell.showsyntaxerror

import logging
logging.basicConfig(filename=path.replace('.log','-exc.log'), format='%(asctime)s %(message)s', level=logging.ERROR, force=True)

import sys
import traceback
import IPython

def showtraceback(self, *args, **kwargs):
    etype, value, tb = self._get_exc_info(kwargs.get('exc_tuple'))
    stb = self.InteractiveTB.structured_traceback(
        etype, value, tb, tb_offset=kwargs.get('tb_offset'))
    logging.error(os.environ.get('JUPYTERHUB_USER') or 'jovyan')
    logging.error(self.InteractiveTB.stb2text(stb))
    _showtraceback(self, *args, **kwargs)

def showsyntaxerror(self, *args, **kwargs):
    etype, value, last_traceback = self._get_exc_info()
    elist = traceback.extract_tb(last_traceback) if kwargs.get('running_compiled_code') else []
    stb = self.SyntaxTB.structured_traceback(etype, value, elist)
    logging.error(os.environ.get('JUPYTERHUB_USER') or 'jovyan')
    logging.error(self.InteractiveTB.stb2text(stb))
    _showsyntaxerror(self, *args, **kwargs)

IPython.core.interactiveshell.InteractiveShell.showtraceback = showtraceback
IPython.core.interactiveshell.InteractiveShell.showsyntaxerror = showsyntaxerror

# 課題: Napierによる乗算のない世界

Step-1) `nest_list()` を使って $R=10^5=10000$ の時のNapierの対数表の元リストを作成する:

In [ ]:
import numpy as np

def nest_list(f, x, c=1):
    while True:
        yield x
        x = f(x)
        if x < c:
            break
    yield x

In [ ]:
napier5=np.array(list(nest_list(lambda l: l*(1-10**(-5)), 10**5, 1)))

In [ ]:
napier5.size, napier5[-1]

In [ ]:
napier5[:100]

In [ ]:
import matplotlib.pyplot as plt

x = np.linspace(0,napier5.size,num=napier5.size)
plt.plot(x,napier5)

Step-2) 上で求めた配列 ```napier5``` を検索し、$R=5$ のときの Napier の対数を求める:

$$
n = \log_{1-10^{-5}}\left( \frac{p_{n}}{10^5} \right)
$$

* [numpy.searchsorted — NumPy v1.23 Manual](https://numpy.org/doc/stable/reference/generated/numpy.searchsorted.html)

昇順に並べ替える (この場合は、逆順にする)

In [ ]:
napier5[::-1][:100]

`numpy.searchsorted()`を使い、指定した値になる添字 (index) を求める

In [ ]:
np.searchsorted(napier5[::-1], 1234), np.searchsorted(napier5[::-1], 5678)

リストを逆順にしたので、正順の際の添字に戻す:

In [ ]:
napier5.size-np.searchsorted(napier5[::-1], 1234), napier5.size-np.searchsorted(napier5[::-1], 56789)

変換がいちいち煩わしいので、正負逆転した配列 (この場合、昇順になる) で `numpy.searchsorted()` を使う

In [ ]:
np.searchsorted(-napier5, -1234), np.searchsorted(-napier5, -56789)

以上をプログラム `napier5_log()` に反映する:

In [ ]:
napier5[0], napier5[-1]

In [ ]:
def napier5_log(x):
    if x < napier5[0] and x > napier5[-1]:
        return np.searchsorted(-napier5, -x)
    else:
        raise IndexError

In [ ]:
napier5_log(1234), napier5_log(56789)

In [ ]:
napier5[439489], napier5[56583]

In [ ]:
((1-10**-5)**439489), ((1-10**-5)**56583)

In [ ]:
import math
math.log(napier5[439489]/10**5, 1-10**-5), math.log(napier5[56583]/10**5, 1-10**-5)

Step-3) 指数法則: $\log(x \times y) = \log(x) + \log(y)$ を使って、掛け算を足け算に変換して計算するプログラムを記述せよ:

In [ ]:
napier5_log(1234), napier5_log(56789)

In [ ]:
napier5_log(1234) + napier5_log(56789)

In [ ]:
napier5[496072]*10**5, 1234*56789

In [ ]:
def napier5_times(x, y):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
napier5_times(1234, 56789), 1234*56789

In [ ]:
math.isclose(napier5_times(1234, 56789), 1234*56789, rel_tol=10**-5)

---
以下、採点用のセルにつき編集できない:

In [ ]:
from nose.tools import assert_equal, assert_true

In [ ]:
assert_true(math.isclose(1234*56789,  napier5_times(1234, 56789) , rel_tol=10**-5))